In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


License Plate Segmentation

In [ ]:
# Create virtual ennv
!apt-get install -y python3-venv

In [ ]:
!python3 -m venv task1
!task1/bin/activate.bat

In [ ]:
!unzip /content/drive/MyDrive/Task_LicensePlateSeg/datasets/licenseplate_80_10_10.zip

In [ ]:
import torch
import torchvision
# to load the dataset
from torch.utils.data import Dataset, DataLoader

In [92]:
# MOBILENET BACKBONE
# model = torchvision.models.segmentation.deeplabv3_mobilenet_v3_large(pretrained=True)
from torchvision.models.segmentation import deeplabv3_mobilenet_v3_large,DeepLabV3_MobileNet_V3_Large_Weights
weights = DeepLabV3_MobileNet_V3_Large_Weights.DEFAULT
model = deeplabv3_mobilenet_v3_large(weights)

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:135: UserWarning: Using 'weights' as positional parameter(s) is deprecated since 0.13 and may be removed in the future. Please use keyword parameter(s) instead.
  warnings.warn(


In [ ]:
# RESNET 50 BACKBONE
from torchvision.models.segmentation import deeplabv3_resnet50,DeepLabV3_ResNet50_Weights
weights = DeepLabV3_ResNet50_Weights.DEFAULT
model = deeplabv3_resnet50(weights)

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:135: UserWarning: Using 'weights' as positional parameter(s) is deprecated since 0.13 and may be removed in the future. Please use keyword parameter(s) instead.
  warnings.warn(
Downloading: "https://download.pytorch.org/models/deeplabv3_resnet50_coco-cd0a2569.pth" to /root/.cache/torch/hub/checkpoints/deeplabv3_resnet50_coco-cd0a2569.pth
100%|██████████| 161M/161M [00:02<00:00, 73.8MB/s]


In [93]:
num_classes = 2  # Set the number of classes : plate and background
batch_size = 8
epochs = 100
data_dir = '/content/licenseplate_80_10_10' #apth to the dataset
loss_fn = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001) # lower the learning rate

transform = weights.transforms(resize_size=None)

# change the classifier at the end of the model
model.classifier[-1] = torch.nn.Conv2d(256, num_classes, kernel_size=(1, 1))

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
model

In [94]:
from PIL import Image
import os

class LicensePlateDataset(Dataset):
  def __init__(self,root_dir,phase,transformer=None):
    self.root_dir = os.path.join(data_dir, phase)
    print(self.root_dir)
    self.image_dir = os.path.join(self.root_dir, 'Images')
    self.mask_dir = os.path.join(self.root_dir, 'SegmentationClass')
    self.image_filenames = os.listdir(self.image_dir)
    self.transformer = transformer

  def __len__(self):
    return len(self.image_filenames)


  def __getitem__(self, index):
    image_path = os.path.join(self.image_dir, self.image_filenames[index])
    mask_path = os.path.join(self.mask_dir, self.image_filenames[index])
    # print(mask_path,image_path)
    
    image = Image.open(image_path).convert('RGB')
    target = Image.open(mask_path).convert('RGB')

    transform_resize = torchvision.transforms.Resize((220,400))
    resized_image,resized_target = transform_resize(image), transform_resize(target)
    
    # Apply any preprocessing
    if self.transformer is not None:
        resized_image = self.transformer(resized_image)
        resized_target = torchvision.transforms.ToTensor()(resized_target)
    
    return resized_image, resized_target

train_dataset = LicensePlateDataset(data_dir,phase='train',transformer=transform)
val_dataset = LicensePlateDataset(data_dir,phase='val',transformer=transform)

train_loader = DataLoader(train_dataset, batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=8)

dataLoader = {'train':train_loader, 'val':val_loader}

/content/licenseplate_80_10_10/train
/content/licenseplate_80_10_10/val


In [ ]:
def train(model,epochs,optimizer,loss_fn,dataLoader,device):
  import copy

  model.to(device)

  best_model_wts = copy.deepcopy(model.state_dict())
  best_loss = 10.0

  for epoch in range(epochs):
      val_loss = 0.0
      train_loss = 0.0

      print('Epoch {}/{}'.format(epoch, epochs - 1))
      
      # Training phase 
      model.train()
      for images, targets in dataLoader['train']:
          images = images.to(device)
          targets = targets.to(device)
          optimizer.zero_grad()
          
          pred = model(images)['out']
          loss = loss_fn(pred, torch.argmax(targets, dim=1)) # get a single channel from the mask

          loss.backward()
          optimizer.step()
          train_loss += loss.item() * images.size(0) # multiplied by batch_size
      
      train_loss /= len(dataLoader['train'].dataset)
      print('Training Loss: {:.4f}'.format(train_loss))
      
      # Validation every 10 epochs
      if (epoch+1) % 10 == 0:
        print("-----------------------------")
        model.eval()
        
        with torch.no_grad():
          for images, targets in dataLoader['val']:
            images = images.to(device)
            targets = targets.to(device)
            pred = model(images)['out']
            loss = loss_fn(pred, torch.argmax(targets, dim=1)) # get a single channel from the mask

            val_loss += loss.item() * images.size(0) # multiplied by batch_size

        val_loss /= len( dataLoader['val'].dataset)
          
        print('Validation Loss: {:.4f}'.format(val_loss))
        
        if best_loss*10000 > val_loss*10000:
            best_loss = val_loss
            best_model_wts = copy.deepcopy(model.state_dict())
            
      print('-' * 10)

  model.load_state_dict(best_model_wts)

  return model,best_loss


model,best_loss = train(model,epochs,optimizer,loss_fn,dataLoader,device)

In [ ]:
best_loss

In [ ]:
torch.save(model,f='/content/drive/MyDrive/Task_LicensePlateSeg/results/model_80_10_10/deeplabv3_mobilenet_v3_large/model_80_10_10')

In [ ]:
torch.save(model,f='/content/drive/MyDrive/Task_LicensePlateSeg/results/model_80_10_10/deeplabv3_resnet50/model_80_10_10')

Performance Metrics
1. IOU for all test images
2. mAP

In [ ]:
!pip install torchmetrics

In [ ]:
# Perfomance metric for test  --- mAP and IOU

def compute_IOU(model, test_loader,device,num_classes):
    from torchmetrics import JaccardIndex
    import numpy as np

    model.eval()

    jaccard = JaccardIndex(task="multiclass", num_classes=num_classes)
    jaccard = jaccard.to(device)

    IOU = []

    with torch.no_grad():
      for images, masks in test_loader:
          images = images.to(device)
          targets = masks.to(device)
          one_channel_masks =torch.argmax(targets, dim=1)
    
          outputs = model(images)['out']
          predictions = torch.argmax(outputs, dim=1)
          # print(predictions.shape,one_channel_masks.shape)
          iou = jaccard(predictions, one_channel_masks)
          iou = iou.detach().cpu().numpy()
          IOU.append(iou)
          # print(iou)

    mean_iou = np.mean(IOU)
    return mean_iou,IOU


test_dataset = LicensePlateDataset(data_dir,phase='test',transformer=transform)
test_loader = DataLoader(test_dataset, batch_size=8)

mean_iou,iou_list = compute_IOU(model,test_loader,device,2)
print("mean IOU---- ",mean_iou)

In [ ]:
import os
import matplotlib.pyplot as plt
from PIL import Image

# Prepare the test images folder
test_folder = '/content/licenseplate_80_10_10/test/Images'

test_batch = torch.zeros( (10,3,220,400), dtype=torch.float32)

# Iterate over the test images
for i,image_file in enumerate(os.listdir(test_folder)):
    # Load and preprocess the image
    image_path = os.path.join(test_folder, image_file)
    image = Image.open(image_path).convert("RGB")

    image_tensor = transform(image)
    test_batch[i]=image_tensor
    image_tensor=image_tensor.unsqueeze(0).to(device)
    # print(image_tensor.shape)
    
    with torch.no_grad():
        outputs = model(image_tensor)['out']
        predicted_masks = torch.argmax(outputs.softmax(dim=1), dim=1)

    # Convert predicted masks to numpy arrays for visualization
    predicted_masks = predicted_masks.squeeze().cpu().numpy()

    plt.figure()
    plt.imshow(image)
    plt.imshow(predicted_masks,alpha=0.6)
    plt.title("pred")
    plt.axis('off')

    plt.show()
    # plt.savefig('/content/drive/MyDrive/Task_LicensePlateSeg/results/model_80_10_10/deeplabv3_mobilenet_v3_large/pred'+str(i)+'.png')

In [ ]:
test_batch= test_batch.to(device)

repetitions=10
total_time = 0
with torch.no_grad():
    for rep in range(repetitions):
        starter, ender = torch.cuda.Event(enable_timing=True), torch.cuda.Event(enable_timing=True)
        starter.record()
        _ = model(test_batch)
        ender.record()
        torch.cuda.synchronize() # wait til the gpu completes
        curr_time = starter.elapsed_time(ender)/1000 #convert to second
        total_time += curr_time
Throughput =   (repetitions*test_batch.size()[0])/total_time
print('Final Throughput: ',Throughput)
print('Inference Time: ',total_time/(repetitions*test_batch.size()[0]))